### Fitting XShooter data with xtool ###

In [1]:
from xtool.data import XShooterData, Order
from xtool.model import OrderModel, VirtualPixelWavelength, GenericBackground, MoffatTrace, SlopedMoffatTrace

from collections import OrderedDict
from scipy import sparse
from scipy import optimize
import numpy as np

#### Reading XShooter data ####

In [2]:
xd = XShooterData('xtool_ds/')

In [3]:
current_order = xd[17]
current_order.enable_flags_as_mask() # important to update the data to include the bad pixels from the pipeline

#### Generating a virtual pixel table for "Wavelength"-pixels ####

In [4]:
virt_pix = VirtualPixelWavelength.from_order(current_order, wavelength_sampling=0.03)
pixel_table = virt_pix()

In [6]:
pixel_table

,pixel_id,wavelength_pixel_id,sub_x,wavelength,slit_pos,normed_wavelength
0,0,4464,0.12,1606.211504,-5.111314,0.999104
1,0,4465,0.56,1606.241504,-5.111314,0.999552
2,0,4466,0.32,1606.271504,-5.111314,1.000000
3,1,4463,0.04,1606.181504,-4.865908,0.998657
4,1,4464,0.44,1606.211504,-4.865908,0.999104
5,1,4465,0.48,1606.241504,-4.865908,0.999552
6,1,4466,0.04,1606.271504,-4.865908,1.000000
7,2,4463,0.32,1606.181504,-4.620392,0.998657
8,2,4464,0.48,1606.211504,-4.620392,0.999104
9,2,4465,0.20,1606.241504,-4.620392,0.999552


#### Initializing the two Models ####

In [7]:
background_mdl = GenericBackground(pixel_table, virt_pix.wavelength_pixels)
trace_mdl = SlopedMoffatTrace(pixel_table, virt_pix.wavelength_pixels)

In [8]:
order_model = OrderModel([background_mdl, trace_mdl])

#### Show fittable parameters ####

In [9]:
order_model

<OrderModel(background_level=[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   9.00733670e+15
   9.00733670e+15   9.00733670e+15], amplitude=[ nan  nan  nan ...,  nan  nan  nan], trace_pos=0.0, trace_slope=0.0, sigma=1.0, beta=1.5 [f])>

#### Initializing the fitter ####

In [10]:
from xtool.model.fitters import Fitter

In [11]:
fitter = Fitter(order_model, current_order)

#### Differential Evolution fit ####

Differential evolution is a very thorough but slow processs. It does not use starting values but moves within the bounds of the parameters. 

In [31]:
order_model.beta.fixed = False
dresult = fitter.fit_differential_evolution(disp=True)

differential_evolution step 1: f(x)= 4.22785e+06
differential_evolution step 2: f(x)= 4.22785e+06
differential_evolution step 3: f(x)= 2.95083e+06
differential_evolution step 4: f(x)= 2.95083e+06
differential_evolution step 5: f(x)= 2.26008e+06
differential_evolution step 6: f(x)= 2.26008e+06
differential_evolution step 7: f(x)= 2.26008e+06
differential_evolution step 8: f(x)= 2.26008e+06
differential_evolution step 9: f(x)= 2.06608e+06
differential_evolution step 10: f(x)= 2.06231e+06
differential_evolution step 11: f(x)= 2.00095e+06
differential_evolution step 12: f(x)= 1.96445e+06
differential_evolution step 13: f(x)= 1.96445e+06
differential_evolution step 14: f(x)= 1.96445e+06
differential_evolution step 15: f(x)= 1.89723e+06
differential_evolution step 16: f(x)= 1.89723e+06
differential_evolution step 17: f(x)= 1.89723e+06
differential_evolution step 18: f(x)= 1.89723e+06
differential_evolution step 19: f(x)= 1.89654e+06
differential_evolution step 20: f(x)= 1.89654e+06
different

In [32]:
dresult

     fun: 1896544.4149950482
 message: 'Optimization terminated successfully.'
    nfev: 1425
     nit: 21
 success: True
       x: array([-2.44188896, -0.26832235,  0.19794569,  2.0046534 ])

#### Nelder Mead / Simplex ####

This one is much faster algorithm that will use starting values, but is unbounded

In [12]:
order_model.trace_pos = -2
order_model.sigma = 0.1
order_model.trace_slope = 0.
order_model.beta.fixed = False
order_model.beta = 2.0
result = fitter.fit_scipy_minimize('Nelder-Mead')

/Users/wkerzend/anaconda3/envs/xtool/lib/python2.7/site-packages/scipy/optimize/_minimize.py:394: RuntimeWarning: Method Nelder-Mead cannot handle constraints nor bounds.
  RuntimeWarning)


Fit finished in 110.747854948 s


/Users/wkerzend/anaconda3/envs/xtool/lib/python2.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:243: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


RuntimeError: Factor is exactly singular

In [20]:
dmatrix, model_widths = order_model.generate_design_matrix(current_order, **OrderedDict([('trace_pos', -2.0), ('trace_slope', 0.0), ('sigma', 0.10000000000000001), ('beta', 2.0)]))

In [13]:
test = order_model.model_list[0]

In [14]:
pixel_table

,pixel_id,wavelength_pixel_id,sub_x,wavelength,slit_pos,normed_wavelength
0,0,4464,0.0,1606.211504,-5.111314,0.999104
1,0,4465,0.0,1606.241504,-5.111314,0.999552
2,0,4466,0.0,1606.271504,-5.111314,1.000000
3,1,4463,0.0,1606.181504,-4.865908,0.998657
4,1,4464,0.0,1606.211504,-4.865908,0.999104
5,1,4465,0.0,1606.241504,-4.865908,0.999552
6,1,4466,0.0,1606.271504,-4.865908,1.000000
7,2,4463,0.0,1606.181504,-4.620392,0.998657
8,2,4464,0.0,1606.211504,-4.620392,0.999104
9,2,4465,0.0,1606.241504,-4.620392,0.999552


In [24]:
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix

#from http://stackoverflow.com/questions/22961541/python-matplotlib-plot-sparse-matrix-pattern

def plot_coo_matrix(m):
    if not isinstance(m, coo_matrix):
        m = coo_matrix(m)
    fig = plt.figure()
    ax = fig.add_subplot(111, axisbg='white')
    ax.plot(m.col, m.row, 's', color='black', ms=1)
    ax.set_xlim(0, m.shape[1])
    ax.set_ylim(0, m.shape[0])
    ax.set_aspect('auto')
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.invert_yaxis()
    ax.set_aspect('auto')
    ax.set_xticks([])
    ax.set_yticks([])
    return ax

#### Plotting in DS9 ####

In [29]:
%matplotlib inline
dmatrix

<83843x8934 sparse matrix of type '<type 'numpy.float64'>'
	with 524722 stored elements in COOrdinate format>

In [33]:
import pyds9

In [34]:
d = pyds9.DS9()

In [35]:
order_model.trace_pos = dresult.x[0]
order_model.trace_slope = dresult.x[1]
order_model.sigma = dresult.x[2]
order_model.beta = dresult.x[3]
model = order_model.evaluate_to_frame(current_order, trace_pos=order_model.trace_pos, 
                                      trace_slope=order_model.trace_slope, sigma=order_model.sigma, beta=order_model.beta.value)
d.set('frame 1')
d.set_np2arr(current_order.data.filled())

d.set('frame 2')
d.set_np2arr(model.filled())

d.set('frame 3')
d.set_np2arr((current_order.data.filled() - model.filled())/current_order.uncertainty.filled())

1